### This file was used o evaluate and understand cuSpatial and cuDF functionality for NTAD Dataset

In [2]:
# Imports used throughout this notebook.
import cuspatial
import cudf
import cupy  
import geopandas as gpd
import os
import pandas as pd
import numpy as np
from shapely.geometry import *
from shapely import wkt

In [ ]:
gdf = gpd.read_file("data/North_American_Roads.shp")

In [4]:
print(gdf.head())

       ID  DIR  LENGTH      LINKID  COUNTRY JURISCODE JURISNAME ROADNUM  \
0   34287    0  201.62    01_34287        1     01_60     Yukon       5   
1   34288    0   42.88    01_34288        1     01_60     Yukon       5   
2   34289    0  185.29    01_34289        1     01_60     Yukon       5   
3  440304    0    1.57  02_2752236        2     02_02    Alaska    None   
4  440305    0    1.41  02_2752403        2     02_02    Alaska    None   

           ROADNAME        ADMIN  SURFACE  LANES  SPEEDLIM  CLASS  NHS  \
0  Dempster Highway  Territorial  Unpaved      2        80      2  3.0   
1  Dempster Highway  Territorial  Unpaved      2        80      2  3.0   
2  Dempster Highway  Territorial  Unpaved      2        80      2  3.0   
3      AHMOAGAK AVE    Municipal    Paved      2        88      5  0.0   
4      AHMOAGAK AVE    Municipal    Paved      2        88      5  0.0   

   BORDER                                           geometry  
0       0  LINESTRING (-138.52554 64.2373

In [5]:
print(gdf.geom_type.unique())

['LineString' 'MultiLineString' None]


In [6]:
# Filtering out None geometries
gdf = gdf[~gdf['geometry'].isna()]

In [7]:
def extract_coords(geom):
    if geom is None:
        return []
    if geom.geom_type == 'LineString':
        return list(geom.coords)
    elif geom.geom_type == 'MultiLineString':
        return [list(line.coords) for line in geom.geoms]
    else:
        return []

# Apply extraction
gdf['coords'] = gdf['geometry'].apply(extract_coords)

In [8]:
#Convert to flattened list of coordinates
def flatten_coords(coords):
    # Handle MultiLineString which returns list of lists
    if coords and isinstance(coords[0], list):
        return [pt for line in coords for pt in line]
    return coords

gdf['flat_coords'] = gdf['coords'].apply(flatten_coords)

In [9]:
print(gdf.head())

       ID  DIR  LENGTH      LINKID  COUNTRY JURISCODE JURISNAME ROADNUM  \
0   34287    0  201.62    01_34287        1     01_60     Yukon       5   
1   34288    0   42.88    01_34288        1     01_60     Yukon       5   
2   34289    0  185.29    01_34289        1     01_60     Yukon       5   
3  440304    0    1.57  02_2752236        2     02_02    Alaska    None   
4  440305    0    1.41  02_2752403        2     02_02    Alaska    None   

           ROADNAME        ADMIN  SURFACE  LANES  SPEEDLIM  CLASS  NHS  \
0  Dempster Highway  Territorial  Unpaved      2        80      2  3.0   
1  Dempster Highway  Territorial  Unpaved      2        80      2  3.0   
2  Dempster Highway  Territorial  Unpaved      2        80      2  3.0   
3      AHMOAGAK AVE    Municipal    Paved      2        88      5  0.0   
4      AHMOAGAK AVE    Municipal    Paved      2        88      5  0.0   

   BORDER                                           geometry  \
0       0  LINESTRING (-138.52554 64.237

#### Separating longitude (x) and latitude (y) values into two flat arrays.

In [10]:
# Separate x and y into lists
x_coords = []
y_coords = []

for coord_list in gdf['flat_coords']:
    for lon, lat in coord_list:
        x_coords.append(lon)
        y_coords.append(lat)

In [11]:
# Convert to cuDF Series for GPU processing
x_series = cudf.Series(x_coords)
y_series = cudf.Series(y_coords)

print(f"Total points loaded to GPU: {len(x_series)}")

Total points loaded to GPU: 25224400


#### Define Bounding Box for Pre-Filtering

In [12]:
# Example bounding box: (minx, miny, maxx, maxy)
bbox_minx, bbox_miny, bbox_maxx, bbox_maxy = -140, 30, -60, 70  # Adjust to your target region

# Create mask to filter points within bounding box
mask = (x_series >= bbox_minx) & (x_series <= bbox_maxx) & \
       (y_series >= bbox_miny) & (y_series <= bbox_maxy)

filtered_x = x_series[mask]
filtered_y = y_series[mask]

print(f"Filtered points within bounding box: {len(filtered_x)}")

Filtered points within bounding box: 14649383


In [14]:
# Example bounding box around a target area
target_lon, target_lat = -75.0, 40.0
offset = 0.05

# Define bounding box corners (in lon/lat degrees)
poly_x = cudf.Series([target_lon - offset, target_lon - offset,
                      target_lon + offset, target_lon + offset,
                      target_lon - offset])
poly_y = cudf.Series([target_lat - offset, target_lat + offset,
                      target_lat + offset, target_lat - offset,
                      target_lat - offset])

# Offsets defining one polygon with one ring
poly_offsets = cudf.Series([0])
ring_offsets = cudf.Series([0])

In [15]:
points_x = cudf.Series(filtered_x.to_numpy())
points_y = cudf.Series(filtered_y.to_numpy())